In [ ]:
%run getSpectrograms.py

In [ ]:
plt.rcParams["image.interpolation"] = 'nearest'
plt.gca().set_title('Spektrogram bez roušky')
plt.gca().set_xlabel('Čas [s]')
plt.gca().set_ylabel('Frekvence [Hz]')
plt.imshow(TMF_trans, origin='lower', aspect='auto', extent = [0 , 1.0, 0 , 8000])
cbar = plt.colorbar()
cbar.set_label('Spektralní hustota výkonu [dB]', rotation=270, labelpad=15)
plt.figure()

plt.gca().set_title('Spektrogram s rouškou')
plt.gca().set_xlabel('Čas [s]')
plt.gca().set_ylabel('Frekvence [Hz]')
plt.imshow(TMN_trans, origin='lower', aspect='auto', extent = [0 , 1.0, 0 , 8000])
cbar = plt.colorbar()
cbar.set_label('Spektralní hustota výkonu [dB]', rotation=270, labelpad=15)

## Funkce implementující DFT

In [ ]:
def dft(source, N):
    result = []
    padded = np.pad(source,(0,N-len(source)))
    for k in range(N):
        sum = np.complex(0,0)
        for n in range(N):
            exponent = -2j * np.pi * n * (k/float(N))
            sum += padded[n] * np.exp(exponent)
        result.append(20*np.log10(np.abs(sum)))

    return result

TMF_cust0 = dft(TMFframes[0],1024)[0:512]
TMF_cust1 = dft(TMFframes[1],1024)[0:512]

plt.gca().set_title('Mask off')
plt.plot(TMF_cust0, label="Vlastní DFT")
plt.plot(TMF_spect[0], label="NumPy")
plt.legend()

plt.figure()
plt.gca().set_title('Mask on')
plt.plot(TMF_cust1, label="Vlastní DFT")
plt.plot(TMF_spect[1], label="NumPy")
plt.legend()

## Výsledek porovnání
NumPy je rychlejší, jinak žádný rozdíl

## H(e^jw)
### Vztah pro výpočet H
H(e^jw) = B(e^jw) / A(e^jw)  
H(e^jw) = (((SUMA i od 0 do 99 z DFT_rámce_Bi)/100) / ((SUMA i od 0 do 99 z DFT_rámce_Ai)/100))


In [ ]:
H_PSD = 20*np.log10(np.abs(H[0:512]))
H = np.divide(TMN_ave,TMF_ave)

plt.gca().set_title('Frekvenční charakteristika rouškového filtru')
plt.gca().set_ylabel('PSD [dB]')
plt.gca().set_xlabel('Frekvence [Hz]')
plt.plot(np.arange(0,Fs/2, Fs/2/512),H_PSD[0:512])

## Krátký komentář k filmu
TODO

# Freqz way
from scipy import signal
#w, H2 = signal.freqz(TMNcorr, TMFcorr, worN=100)
#plt.plot(H2)
f, t, sgr = signal.spectrogram(TMNcent, 16000, nperseg=320, nfft=1024)
# prevod na PSD
# (ve spektrogramu se obcas objevuji nuly, ktere se nelibi logaritmu, proto +1e-20)
sgr_log = 10 * np.log10(sgr+1e-20) 
plt.figure(figsize=(9,3))
plt.imshow(sgr_log, origin='lower', aspect='auto', extent = [0 , 1.0, 0 , 8000])
plt.gca().set_xlabel('Čas [s]')
plt.gca().set_ylabel('Frekvence [Hz]')
cbar = plt.colorbar()
cbar.set_label('Spektralní hustota výkonu [dB]', rotation=270, labelpad=15)

plt.tight_layout()